In [1]:
# import libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA    
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
import numpy as np
from tqdm import tqdm
from random import randint
from time import sleep

# 1. Dataset: ASOS

In [4]:
# reading the data
pd.set_option('display.max_columns', None)

In [6]:
df_asos = pd.read_csv('data/products_asos.csv')

In [7]:
df_asos

,url,name,size,category,price,color,sku,description,images
0,https://www.asos.com/stradivarius/stradivarius...,New Look trench coat in camel,"UK 4,UK 6,UK 8,UK 10,UK 12,UK 14 - Out of stoc...",New Look trench coat in camel,49.99,Neutral,126704571.0,[{'Product Details': 'Coats & Jackets by New L...,['https://images.asos-media.com/products/new-l...
1,https://www.asos.com/stradivarius/stradivarius...,New Look trench coat in camel,"UK 4,UK 6,UK 8,UK 10,UK 12,UK 14 - Out of stoc...",New Look trench coat in camel,49.99,Neutral,126704571.0,[{'Product Details': 'Coats & Jackets by New L...,['https://images.asos-media.com/products/new-l...
2,https://www.asos.com/asos-design/asos-design-l...,New Look trench coat in camel,"UK 4,UK 6,UK 8,UK 10,UK 12,UK 14 - Out of stoc...",New Look trench coat in camel,49.99,Neutral,126704571.0,[{'Product Details': 'Coats & Jackets by New L...,['https://images.asos-media.com/products/new-l...
3,https://www.asos.com/new-look/new-look-trench-...,New Look trench coat in camel,"UK 4,UK 6,UK 8,UK 10,UK 12,UK 14 - Out of stoc...",New Look trench coat in camel,49.99,Neutral,126704571.0,[{'Product Details': 'Coats & Jackets by New L...,['https://images.asos-media.com/products/new-l...
4,https://www.asos.com/stradivarius/stradivarius...,Stradivarius double breasted wool coat in grey,"XS - UK 6,S - UK 8,M - UK 10,L - UK 12,XL - UK 14",Stradivarius double breasted wool coat in grey,59.99,GREY,123650194.0,[{'Product Details': 'Coats & Jackets by Strad...,['https://images.asos-media.com/products/strad...
...,...,...,...,...,...,...,...,...,...
30840,https://www.asos.com/urban-revivo/urban-revivo...,Urban Revivo square neck mini dress in floral ...,"XS - UK 6 - Out of stock,S - UK 8 - Out of sto...",Urban Revivo square neck mini dress in floral ...,44.00,Multi,116745746.0,[{'Product Details': 'Mini dress by Urban Revi...,['https://images.asos-media.com/products/urban...
30841,https://www.asos.com/asos-design/asos-design-l...,ASOS DESIGN long sleeve maxi t-shirt dress in ...,"UK 4 - Out of stock,UK 6 - Out of stock,UK 8 -...",ASOS DESIGN long sleeve maxi t-shirt dress in ...,24.00,Black,1444255.0,[{'Product Details': 'Dress by ASOS DESIGN Act...,['https://images.asos-media.com/products/asos-...
30842,https://www.asos.com/asyou/asyou-layered-t-shi...,ASYOU layered t-shirt dress with focus graphic...,"UK 4 - Out of stock,UK 6,UK 8 - Out of stock,U...",ASYOU layered t-shirt dress with focus graphic...,22.99,Washed black,110783769.0,[{'Product Details': 'Dress by ASYOU Exclusive...,['https://images.asos-media.com/products/asyou...
30843,https://www.asos.com/miss-selfridge/miss-selfr...,Miss Selfridge Petite rib knit frill hem funne...,"UK 4 - Out of stock,UK 6 - Out of stock,UK 8 -...",Miss Selfridge Petite rib knit frill hem funne...,32.99,BLACK,116363729.0,[{'Product Details': 'Petite by Miss Selfridge...,['https://images.asos-media.com/products/miss-...


### Cleaning the data

In [5]:
#CREATING A FUNCTION TO DO ALL THE CLEANING IN THE DATAFRAME


def asos_cleaner(x):

    #DROPPINF 18 NANS
    x = x.dropna().reset_index(drop = True)
    
    #TREATING URL COLUMN

    x['url'].str.split('/',expand=True)
    x['brand'] = x['url'].str.split('/',expand=True)[3]
    x['iteminfo'] = x['url'].str.split('/',expand=True)[4]
    
    #CREATE TEMPORARY COLUMN TO BE ABLE TO DEAL WITH PROBLEM EMPTY VALUES LATER ON#

    #x['iteminfo_temp'] = x['iteminfo'].str.split('in-',expand=True)[0]
    x['iteminfo'] = x['iteminfo'].str.split('in-',expand=True)[0]
    
    
    info = []
    for i in range(len(x)):
        if x['brand'][i] in x['iteminfo'][i]:
            info.append(x['iteminfo'][i].replace(x['brand'][i],''))
        else:
            info.append(x['iteminfo'][i])
    x['iteminfo'] = info
    
    
    #EXTRACTING THE TYPE OF CLOTHES FROM THE COLUMN 'ITEMINFO'

    x['iteminfo'].str.replace('-',' ')
    
    #TAKING OFF THE WORD 'PETITE' FROM 'DESCRIPTION' COLUMN. PUTTIN THE REST OF VALUES IN A NEW COLUMN 'PROD'

    prod = []
    petite = []
    
    for i in range(len(x['description'])):
        if 'petite' in x['description'][i]:
            petite.append(x['description'][i])
        prod.append(x['description'][i].split(' by')[0].split(':')[1])
        
    x['productinfo'] = prod
    
    
    #AFTER CHECKING THE 'PRODUCTINFO' COLUMN, I SEE THAT THERE ARE SOME VALUES WITH THE SAME PATERN THAT I DON'T WANT (PLUS-SIZE, TALL, MATERNITY), WE'RE GOING TO REDUCE THE LIST SUBSTITUTING THAT VALUES FOR AN EMPTY SPACE: 


    reductionproduct = []
    for i in x['productinfo']:
        if 'Plus-size' in i:
            reductionproduct.append(i.replace('Plus-size',''))
        elif 'Plus' in i:
            reductionproduct.append(i.replace('Plus',''))
        elif 'Tall' in i:
            reductionproduct.append(i.replace('Tall',''))
        elif 'Maternity' in i:
            reductionproduct.append(i.replace('Maternity',''))
        elif 'Petite' in i:
            reductionproduct.append(i.replace('Petite',''))     
        else:
            reductionproduct.append(i)
        
    x['productinfo'] = reductionproduct
    
    #CLEANING MORE DETAILS FROM THAT COLUMN

    x['productinfo'] = x['productinfo'].str.strip()
    x['productinfo'] = x['productinfo'].str.strip('"')
    x['productinfo'] = x['productinfo'].str.strip("'")
    x['productinfo'] = x['productinfo'].str.lstrip()
    x['productinfo'] = x['productinfo'].str.rstrip()
    x['productinfo'] = x['productinfo'].str.replace("And",'&')
    x['productinfo'] = x['productinfo'].str.lower()
    
    x = x[x['productinfo'] != 'curve &  size'].reset_index(drop = True)
    
    #I'M GOING TO CREATE A NEW COLUMN THAT I WILL USE TO DO MY HASHTAG RESEARCH ON IG LATER.

    x['#search'] = x['productinfo']
    
    
    #CREATE A LOOP TO BE ABLE TO IDENTIFY BLANK VALUES IN THE ITEM INFO COLUMN
    indicesspace = []
    for i in range(len(x['iteminfo'])):
        if len(x['iteminfo'][i]) < 1:
            indicesspace.append(i)
            
    #CREATE A FUNCTION TO CLEAN THIS 'BLANK VALUES'
        
    def productinfoclean(y):
    
        #CREATE A DATAFRAME WITH THE INDICES OF THE BLANK VALUES
        empties = pd.DataFrame(x['url'].str.split('/',expand=True)[4].loc[y])
        
        
        #APPEND THE CORRECTED VALUES TO THE LIST
        replacement = []
        for i in empties[4]:
            if '-jacket' in i:
                replacement.append('jacket')
            elif 'coat' in i:
                replacement.append('coat')
            elif 'cardi' in i:
                replacement.append('cardigan')
            elif 'jumper' in i:
                replacement.append('jumper')
            elif 'co-ord' in i:
                replacement.append('co-ord')
            elif 'midi-dress' in i:
                replacement.append('mididress')
            elif 'mini-dress' in i:
                replacement.append('minidress')
            elif 'maxi-dress' in i:
                replacement.append('maxidress')
            elif 'dress' in i:
                replacement.append('dress')
            elif 'lingerie' in i:
                replacement.append('lingerie')
            elif 'pyjama' in i:
                replacement.append('pyjamas')
            elif 'vest' in i:
                replacement.append('vest')
            elif 'bodysuit' in i:
                replacement.append('bodysuit')
            elif 'body' in i:
                replacement.append('body')
            elif 'crop-top' in i:
                replacement.append('crop tops')
            elif 'corset-top' in i:
                replacement.append('corset')
            elif 'cami' in i:
                replacement.append('cami')
            elif 't-shirt' in i:
                replacement.append('tshirt')
            elif 'shirt' in i:
                replacement.append('shirt')
            elif 'tunic' in i:
                replacement.append('tunic')
            elif 'jean' in i:
                replacement.append('jean')
            elif 'trouser' in i:
                replacement.append('trousers')
            elif 'flare' in i:
                replacement.append('flare jeans')
            elif 'jumpsuit' in i:
                replacement.append('jumpsuit')
            elif 'midi-skirt' in i:
                replacement.append('midiskirt')
            elif 'blazer' in i:
                replacement.append('blazer')
            elif 'top' in i:
                replacement.append('top')
            else:
                replacement.append(i)
        
        
        # ASSIGN VALUES TO 
        empties['category'] = replacement
           
      
        #list(empties.index)
        return empties[4].loc[list(empties.index)], empties['category'].loc[list(empties.index)]
    
    
    #USING THE FUNCTION TO CLEAN THE 'BLANK VALUES'
    
    x['iteminfo'].loc[indicesspace], x['productinfo'].loc[indicesspace] = productinfoclean(indicesspace)
    
    
    #IDENTIFYING THE INDEXES WERE THE PROD INFO WAS IMPOSSIBLE TO EXTRACT
    
    productinfospace = []
    for i in range(len(x['productinfo'])):
        if len(x['productinfo'][i]) < 1:
            productinfospace.append(i)
    
    #DELETING THOSE VALUES
    x = x.drop(productinfospace).reset_index(drop = True)
    
    
    #NOW I'M GOING TO CHANGE SOME OF THE NAMES OF THE TYPES OF CLOTHES, SO LATER, I CAN DO THE MATCH WITH THE HASHTAGS ON IG
    
    x['#search'].replace('dresses', 'dress', inplace=True)
    x['#search'].replace('tops', 'top', inplace=True)
    x['#search'].replace('jumpers & cardigans', 'cardigan', inplace=True)
    x['#search'].replace('lingerie & nightwear', 'nightwear', inplace=True)
    x['#search'].replace('trousers & leggings', 'trousers', inplace=True)
    x['#search'].replace('mini dress', 'minidress', inplace=True)
    x['#search'].replace('coats & jackets', 'coat', inplace=True)
    x['#search'].replace('t-shirt', 'tshirt', inplace=True)
    x['#search'].replace('midi dress', 'mididress', inplace=True)
    x['#search'].replace('blazers', 'blazer', inplace=True)
    x['#search'].replace('t-shirts', 'tshirt', inplace=True)
    x['#search'].replace('maxi dress', 'maxidress', inplace=True)
    x['#search'].replace('skirts', 'skirt', inplace=True)
    x['#search'].replace('jackets', 'jacket', inplace=True)
    x['#search'].replace('swimwear & beachwear', 'swimwear', inplace=True)
    x['#search'].replace('pyjama set', 'pyjamas', inplace=True)
    x['#search'].replace('shirts', 'shirt', inplace=True)
    x['#search'].replace('vests', 'vest', inplace=True)
    x['#search'].replace('coats', 'coat', inplace=True)
    x['#search'].replace('jumpers', 'jumper', inplace=True)
    x['#search'].replace('cardigans', 'cardigan', inplace=True)
    x['#search'].replace('denim jacket', 'denimjacket', inplace=True)
    x['#search'].replace('sports top', 'sportstop', inplace=True)
    x['#search'].replace('beach dress', 'beachdress', inplace=True)
    x['#search'].replace('shirt dress', 'shirtdress', inplace=True)
    x['#search'].replace('mini skirt', 'miniskirt', inplace=True)
    x['#search'].replace('hoodies & sweatshirts', 'hoodie', inplace=True)
    x['#search'].replace('corset top', 'corsettop', inplace=True)
    x['#search'].replace('cami top', 'camitop', inplace=True)
    x['#search'].replace('blouses', 'blouse', inplace=True)
    x['#search'].replace('fuller-bust bra', 'fullerbustbra', inplace=True)
    x['#search'].replace('t-shirt dress', 'tshirtdress', inplace=True)
    x['#search'].replace('bras', 'bra', inplace=True)
    x['#search'].replace('jumpsuits & playsuits', 'jumpsuit', inplace=True)
    x['#search'].replace('sports bra', 'sportsbra', inplace=True)
    x['#search'].replace('thong multipack', 'nightwear', inplace=True)
    x['#search'].replace('flares', 'flaretrousers', inplace=True)
    x['#search'].replace('thong', 'nightwear', inplace=True)
    x['#search'].replace('thongs', 'nightwear', inplace=True)
    x['#search'].replace('co-ords', 'coord', inplace=True)
    x['#search'].replace('tank tops', 'tanktop', inplace=True)
    x['#search'].replace('bralets', 'bralet', inplace=True)
    x['#search'].replace('night dress', 'nightdress', inplace=True)
    x['#search'].replace('tracksuit set', 'tracksuit', inplace=True)
    x['#search'].replace('briefs multipack', 'brief', inplace=True)
    x['#search'].replace('co-ord set', 'coord', inplace=True)
    x['#search'].replace('gilets', 'gilet', inplace=True)
    x['#search'].replace('pyjama top', 'pyjamas', inplace=True)
    x['#search'].replace('beach cover-up', 'beachcoveup', inplace=True)
    x['#search'].replace('denim shirt', 'denimshirt', inplace=True)
    x['#search'].replace('corset tops', 'corsettop', inplace=True)
    x['#search'].replace('t-shirt multipack', 'tshirt', inplace=True)
    x['#search'].replace('polo top', 'poloshirt', inplace=True)
    x['#search'].replace('polo shirts', 'poloshirt', inplace=True)
    x['#search'].replace('vest multipack', 'vest', inplace=True)
    x['#search'].replace('bikini top', 'bikini', inplace=True)
    x['#search'].replace('bra multipack', 'bra', inplace=True)
    x['#search'].replace('loungewear set', 'loungewear', inplace=True)
    x['#search'].replace('nipple covers', 'nipplecovers', inplace=True)
    x['#search'].replace('shaping shorts', 'shapingshorts', inplace=True)
    x['#search'].replace('bikini briefs', 'bikinibrief', inplace=True)
    x['#search'].replace('suspender belt', 'suspenderbelt', inplace=True)
    x['#search'].replace('lingerie accessory', 'lingerie', inplace=True)
    x['#search'].replace('top multipack', 'top', inplace=True)
    x['#search'].replace('football shirts', 'footballshirt', inplace=True)
    x['#search'].replace('pyjama bottoms', 'pyjamas', inplace=True)
    x['#search'].replace('legging shorts', 'leggingshort', inplace=True)
    x['#search'].replace('socks & tights', 'socks', inplace=True)
    x['#search'].replace('hoodies', 'hoodie', inplace=True)
    x['#search'].replace('lingerie bralets', 'bralet', inplace=True)
    x['#search'].replace('tracksuits', 'tracksuit', inplace=True)
    x['#search'].replace('leather jacket', 'leatherjacket', inplace=True)
    x['#search'].replace('midi skirt', 'midiskirt', inplace=True)
    x['#search'].replace('lingerie bodies', 'lingerie', inplace=True)
    x['#search'].replace('underwear set', 'underwear', inplace=True)
    x['#search'].replace('underwear', 'suspenderbelt', inplace=True)
    x['#search'].replace('bikini bottoms', 'bikini', inplace=True)
    x['#search'].replace('flare jeans', 'flarejeans', inplace=True)
    x['#search'].replace('denim skirt', 'denimskirt', inplace=True)
    x['#search'].replace('beach shirt', 'beachshirt', inplace=True)
    x['#search'].replace('jumpsuits', 'jumpsuit', inplace=True)
    x['#search'].replace('product code', 'bathproduct', inplace=True)
    x['#search'].replace('nursing bra', 'nursingbra', inplace=True)
    x['#search'].replace('loungewear sets', 'loungewear', inplace=True)
    x['#search'].replace('plunge bra', 'plungebra', inplace=True)
    x['#search'].replace('power short', 'shorts', inplace=True)
    x['#search'].replace('fuller-bust bra multipack', 'fullerbustbra', inplace=True)
    x['#search'].replace('pyjama tops', 'pyjamas', inplace=True)
    x['#search'].replace('bralet multipack', 'bralet', inplace=True)
    x['#search'].replace('dressing gown', 'dressinggown', inplace=True)
    x['#search'].replace('sleep set', 'pyjamas', inplace=True)
    x['#search'].replace('tank', 'nightwear', inplace=True)
    x['#search'].replace('bra pack', 'bra', inplace=True)
    x['#search'].replace('fuller bust dress', 'fullerbustdress', inplace=True)
    x['#search'].replace('nightwear sets', 'nightwear', inplace=True)
    x['#search'].replace('twin set', 'coord', inplace=True)
    x['#search'].replace('blouse and saree set', 'Saree', inplace=True)
    x['#search'].replace('sweatshirts', 'sweatshirt', inplace=True)
    x['#search'].replace('denim shorts', 'denimshorts', inplace=True)
    x['#search'].replace('wide-leg jeans', 'widelegjeans', inplace=True)
    x['#search'].replace('maxi skirt', 'maxiskirt', inplace=True)
    x['#search'].replace('leggings shorts', 'leggingshort', inplace=True)
    x['#search'].replace('bikini tops', 'bikini', inplace=True)
    x['#search'].replace('suspender set', 'suspender', inplace=True)
    x['#search'].replace('t-shirt pack', 'tshirt', inplace=True)
    x['#search'].replace('denim vest', 'denimvest', inplace=True)
    x['#search'].replace('bum bag', 'bumbag', inplace=True)
    x['#search'].replace('fuller-bust sports bra', 'sportsbra', inplace=True)
    x['#search'].replace('nighties', 'nightie', inplace=True)
    x['#search'].replace('fuller-bust pyjama set', 'pyjamas', inplace=True)
    x['#search'].replace('night tee', 'nighttshirt', inplace=True)
    x['#search'].replace('yoga pants', 'yogapant', inplace=True)
    x['#search'].replace('lounge joggers', 'loungejoggers', inplace=True)
    x['#search'].replace('gym bra', 'gymbra', inplace=True)
    x['#search'].replace('ski jacket', 'skijacket', inplace=True)
    x['#search'].replace('suits & separates', 'suit', inplace=True)
    x['#search'].replace('suits', 'suit', inplace=True)
    x['#search'].replace('kimonos', 'kimono', inplace=True)
    
    
    #I WANT TO HAVE A COLUMN WITH THE GENERAL CATEGORY OF ALL THE TYPES OF CLOTHES I HAVE EXTRACT BEFORE, TO USE IT LATER
    
    categories = {'coat': "coats", 
    'jacket' : "jackets",
    'shacket' :"jackets",
    'gilet':"jackets",
    'denimjacket':"jackets",
    'raincoat':"coats",
    'skijacket': "jackets",
    'denimvest':"jackets",
    'sweatshirt':"blouses&shirts",
    'parka': "coats",
    'shirt': "blouses&shirts",
    'vest':"jackets",
    'blazer':"jackets",
    'leatherjacket': "jackets",
    'waistcoat':"coats",
    'coord':"sport",
    'cardigan':"cardigans",
    'jumper': "jumpers",
    'top': "tshirts",
    'bathproduct':"complements",
    'hoodie':"jumpers",
    'nightwear':"nightwear",
    'blouse': "blouses&shirts",
    'loungewear': "pyjamas",
    'camitop': "tshirts",
    'dress':"dresses",
    'mididress':"dresses",
    'shirtdress':"dresses",
    'tshirtdress':"dresses",
    'maxidress':"dresses",
    'minidress':"dresses",
    'shorts': "trousers",
    'swimwear': "beach",
    'beachdress':"dresses",
    'tshirt':"tshirts",
    'Saree': "dresses",
    'jumpsuit': "suits",
    'fullerbustdress':"dresses",
    'miniskirt': "skirts",
    'fullerbustbra':"bras",
    'bralet':"bras",
    'pyjamas': "pyjamas",
    'nipplecovers': "nightwear",
    'lingerie': "nightwear",
    'bra':"bras",
    'body': "tshirts",
    'shapingshorts': "trousers",
    'nightdress':"dresses",
    'bodysuit': "suits",
    'brief': "nightwear",
    'plungebra':"bras",
    'briefs': "nightwear",
    'sportsbra':"bras",
    'socks': "complements",
    'nightie': "pyjamas",
    'suspender belts': "nightwear",
    'stick-on cups': "nightwear",
    'playsuit': "suits",
    'shapewear': "nightwear",
    'suspenderbelt': "nightwear",
    'magic lift solution':"nightwear",
    'pyjama romper': "pyjamas",
    'onesie': "pyjamas",
    'accessories': "complements",
    'dressinggown': "pyjamas",
    'briefs pack': "nightwear",
    'suspender': "complements",
    'brief multipack': "nightwear",
    'corset': "nightwear",
    'nursingbra':"bras",
    'teddy': "jumpers",
    'multipack bra':"bras",
    'robe': "accesories",
    'kimono': "nightwear",
    'nighttshirt': "pyjamas",
    'stockings': "complements",
    'higher power shorts': "trousers",
    'thigh tape': "accesories",
    'gymbra':"bras",
    'leggings': "trousers",
    'sportswear': "sport",
    'tracksuit': "suits",
    'joggers': "trousers",
    'sweatpants': "trousers",
    'flaretrousers': "trousers",
    'trousers': "trousers",
    'yogapant': "trousers",
    'loungejoggers': "trousers",
    'crop tops':"tshirts",
    'minitop':"tshirts",
    'bodies': "tshirts",
    'sportstop':"tshirts",
    'corsettop':"tshirts",
    'camis':"tshirts",
    'denimshirt':"blouses&shirts",
    'tunics': "jackets",
    'beachcoveup': "beach",
    'tanktop':"tshirts",
    'top pack':"tshirts",
    'poloshirt': "blouses&shirts",
    'skirt': "skirts",
    'workwear dresses':"dresses",
    'nursing camisole pack': "shirt" ,
    'footballshirt':"tshirts",
    'beachshirt': "blouses&shirts",
    'suit': "suits",
    'jeans': "trousers",
    'flarejeans': "trousers",
    'bikini': "beach",
    'skort': "skirts",
    'bikinibrief': "nightwear",
    'denimshorts': "trousers",
    'widelegjeans': "trousers",
    'maxiskirt': "skirts",
    'midiskirt': "skirts",
    'leggingsshorts': "trousers",
    'leggingshort': "trousers",
    'denimskirt': "skirts",
    'fleece': "sport",
    'beanie': "complements",
    'bumbag': "complements",
    'bracelet':"complements",
    'swimsuit':"beach"}
    
    x['categories'] = x['#search'].map(categories)
    
    
    #HERE I'M GOING TO CLEAN THE "COLOR" COLUMN TO MAKE IT MORE SIMPLE
    
    x['color'] = x['color'].str.lower()
    
    colours = ['black', 'white', 'red', 'yellow', 'orange', 'pink', 'green', 'blue',  'purple', 'grey', 'brown', 'beige', 'gold', 'silver', 'violet',  'khaki', 'coral', 'navy', 'mint', 'turquoise', 'olive', 'crimson', 'ivory', 'lavender', 'maroon', 'leopardprint', 'floral']
    for i in range(len(x['color'])):
        found = False  # Flag to check if any color from 'colours' is found in 'i'
        # Iterate through the 'colours' list
        for j in colours:
            if j in x['color'][i]:
                x['color'][i] = j
                #inlist.append(j)
                found = True  # Set the flag to True if a color is found
        # If no color from 'colours' is found in 'i', append 'i' to 'notinlist'
        if not found:
            x['color'][i] = x['color'][i]
            
            
            
    #I'M GOING TO CREATE A NEW COLUMN WITH THE COLOURS I WILL USE TO DO MY RESEARCH ON IG LATER.

    x['#colour'] = x['color']
    
    
    
    #NOW I'M GOING TO TREAT THE VALUES THAT ARE NOT IN MY SIMPLE LIST OF COLOURS MANUALLY.

    x['#colour'].replace('charcoal', 'black', regex=True, inplace=True)
    x['#colour'].replace('rust', 'brown', regex=True, inplace=True)
    x['#colour'].replace('lime', 'yellow', regex=True, inplace=True)
    x['#colour'].replace('blush', 'pink', regex=True, inplace=True)
    x['#colour'].replace('neutral', 'beige', regex=True, inplace=True)
    x['#colour'].replace('taupe', 'grey', regex=True, inplace=True)
    x['#colour'].replace('burgundy', 'red', regex=True, inplace=True)
    x['#colour'].replace('oatmeal', 'beige', regex=True, inplace=True)
    x['#colour'].replace('tan', 'brown', regex=True, inplace=True)
    x['#colour'].replace('wine', 'red', regex=True, inplace=True)
    x['#colour'].replace('mono', 'black', regex=True, inplace=True)
    x['#colour'].replace('floral print', 'floral', regex=True, inplace=True)
    x['#colour'].replace('cobalt', 'blue', regex=True, inplace=True)
    x['#colour'].replace('teal', 'turquoise', regex=True, inplace=True)
    x['#colour'].replace('mustard', 'yellow', regex=True, inplace=True)
    x['#colour'].replace('rose', 'pink', regex=True, inplace=True)
    x['#colour'].replace('animal print', 'leopardprint', regex=True, inplace=True)
    x['#colour'].replace('natural', 'beige', regex=True, inplace=True)
    x['#colour'].replace('mink', 'grey', regex=True, inplace=True)
    x['#colour'].replace('mauve', 'violet', regex=True, inplace=True)
    x['#colour'].replace('sand', 'brown', regex=True, inplace=True)
    x['#colour'].replace('leopard print', 'leopardprint', regex=True, inplace=True)
    x['#colour'].replace('emerald', 'green', regex=True, inplace=True)
    x['#colour'].replace('check', 'black', regex=True, inplace=True)
    x['#colour'].replace('plum', 'violet', regex=True, inplace=True)
    x['#colour'].replace('leopard', 'leopardprint', regex=True, inplace=True)
    x['#colour'].replace('peach', 'orange', regex=True, inplace=True)
    x['#colour'].replace('champagne', 'yellow', regex=True, inplace=True)
    x['#colour'].replace('monochrome', 'black', regex=True, inplace=True)
    x['#colour'].replace('zebra print', 'leopardprint', regex=True, inplace=True)
    x['#colour'].replace('mocha', 'brown', regex=True, inplace=True)
    x['#colour'].replace('oyster', 'beige', regex=True, inplace=True)
    x['#colour'].replace('lemon', 'yellow', regex=True, inplace=True)
    x['#colour'].replace('washed ', '', regex=True, inplace=True)
    
    
    #TREATING SOME MORE VALUES TO HAVE A BIT MORE SIMPLE LIST OF COLOURS. 

    x['#colour'].replace('multi', 'pattern', regex=True, inplace=True)
    x['#colour'].replace('cream', 'beige', regex=True, inplace=True)
    x['#colour'].replace('khaki', 'beige', regex=True, inplace=True)
    x['#colour'].replace('navy', 'blue', regex=True, inplace=True)
    x['#colour'].replace('lilac', 'purple', regex=True, inplace=True)
    x['#colour'].replace('stone', 'grey', regex=True, inplace=True)
    x['#colour'].replace('ivory', 'white', regex=True, inplace=True)
    x['#colour'].replace('camel', 'brown', regex=True, inplace=True)
    x['#colour'].replace('ecru', 'beige', regex=True, inplace=True)
    x['#colour'].replace('sage', 'green', regex=True, inplace=True)
    x['#colour'].replace('chocolate', 'brown', regex=True, inplace=True)
    x['#colour'].replace('olive', 'green', regex=True, inplace=True)
    x['#colour'].replace('violet', 'purple', regex=True, inplace=True)
    x['#colour'].replace('leopardprintprint', 'leopardprint', regex=True, inplace=True)
    x['#colour'].replace('mint', 'green', regex=True, inplace=True)
    x['#colour'].replace('coral', 'pink', regex=True, inplace=True)
    
    
    #I NEED TO DEAL WITH THE NANS I HAVE IN CATEGORIES, THERE ARE ONLY 9, SO WE WILL DROP THEM
    x.dropna(inplace = True)
    
    #NEW DF WITH JUST THE INFO I'M GOING TO CHOOSE FOR IG. THE ONES WITH # WILL BE THE ONES TO DO THE COMBINATION TO LOOK FOT HASHTAGS. 
    x = x[['name', 'price', 'brand', 'productinfo', 'categories', '#search', '#colour']].reset_index(drop = True)
    
    #TAKING OF THE SPACES IN BRAND COLUMN
    x['#brand'] = x['brand'].str.replace('-', ' ')
    
    reductionbrand = []
    for i in x['#brand']:
        if 'plus-size' in i:
            reductionbrand.append(i.replace('Plus-size',''))
        elif 'plus' in i:
            reductionbrand.append(i.replace('Plus',''))
        elif 'tall' in i:
            reductionbrand.append(i.replace('Tall',''))
        elif 'maternity' in i:
            reductionbrand.append(i.replace('Maternity',''))
        elif 'petite' in i:
            reductionbrand.append(i.replace('Petite',''))     
        else:
            reductionbrand.append(i)
        
    x['#brand'] = reductionbrand
    
    #NOW, I'M GOING TO CREATE A NEW COLUMN MIXING SOME COLUMNS. I WILL USE THAT INFO TO LOOK FOR HASHTAGS ON IG
    x['#1'] = x['#colour']+x['#search']
    x['#2'] = x['#brand'].replace(' ','',regex=True)+x['#search']
    
    searchlistold = list(x['#1'].unique())+list(x['#2'].unique())
    
    searchlist = []
    for i in searchlistold:
        searchlist.append(i.replace(" ",""))
    
    return x, searchlist

In [6]:
asos, hashsearch = asos_cleaner(df_asos)

/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_1764/3102896834.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['iteminfo'].loc[indicesspace], x['productinfo'].loc[indicesspace] = productinfoclean(indicesspace)
/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_1764/3102896834.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['iteminfo'].loc[indicesspace], x['productinfo'].loc[indicesspace] = productinfoclean(indicesspace)
/var/folders/sm/8s2c1j894rq6v788yfhxt27r0000gn/T/ipykernel_1764/3102896834.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
asos.head(20)

,name,price,brand,productinfo,categories,#search,#colour,#brand,#1,#2
0,New Look trench coat in camel,49.99,stradivarius,coats & jackets,coats,coat,beige,stradivarius,beigecoat,stradivariuscoat
1,New Look trench coat in camel,49.99,stradivarius,coats & jackets,coats,coat,beige,stradivarius,beigecoat,stradivariuscoat
2,New Look trench coat in camel,49.99,asos-design,coats & jackets,coats,coat,beige,asos design,beigecoat,asosdesigncoat
3,New Look trench coat in camel,49.99,new-look,coats & jackets,coats,coat,beige,new look,beigecoat,newlookcoat
4,Stradivarius double breasted wool coat in grey,59.99,stradivarius,coats & jackets,coats,coat,grey,stradivarius,greycoat,stradivariuscoat
5,Stradivarius double breasted wool coat in grey,59.99,stradivarius,coats & jackets,coats,coat,grey,stradivarius,greycoat,stradivariuscoat
6,Stradivarius double breasted wool coat in grey,59.99,barneys-originals,coats & jackets,coats,coat,grey,barneys originals,greycoat,barneysoriginalscoat
7,Stradivarius double breasted wool coat in grey,59.99,stradivarius,coats & jackets,coats,coat,grey,stradivarius,greycoat,stradivariuscoat
8,JDY oversized trench coat in stone,45.00,asos-petite,coats & jackets,coats,coat,grey,asos petite,greycoat,asospetitecoat
9,JDY oversized trench coat in stone,45.00,jdy,coats & jackets,coats,coat,grey,jdy,greycoat,jdycoat


In [8]:
hashsearch

['beigecoat',
 'greycoat',
 'pinkcoat',
 'blackcoat',
 'purplecoat',
 'blackjacket',
 'beigejacket',
 'bluecoat',
 'browncoat',
 'greenshacket',
 'dirtydarkwashcoat',
 'bluejacket',
 'brownjacket',
 'greencoat',
 'midwashjacket',
 'chalkjacket',
 'coffeecoat',
 'cementcoat',
 'blackgilet',
 'darkturquoisejacket',
 'whitecoat',
 'blackshacket',
 'denimdarkcoat',
 'pinkjacket',
 'patterntweedcoat',
 'bluedenimjacket',
 'whitejacket',
 'herringbonecoat',
 'hotfudgecoat',
 'beigegilet',
 'fossiljacket',
 'bleachdenimjacket',
 'redcoat',
 'silvercoat',
 'patterncoat',
 'yellowcoat',
 'purplegilet',
 'oatcoat',
 'cypresscoat',
 'birchcoat',
 'leopardprintjacket',
 'fawnjacket',
 'greenjacket',
 'patterngilet',
 'blackchromecoat',
 'brownandbeigecoat',
 'greyjacket',
 'whitedenimjacket',
 'orangecoat',
 'amethystsmokecoat',
 'yellowjacket',
 'brownraincoat',
 'beetrootskijacket',
 'whitedenimvest',
 'frenchorchidcoat',
 'laureloakcoat',
 'fossilcoat',
 'caneelbaycoat',
 'patternjacket',
 'och

# 2. Scraping from IG

In [11]:
#CREATING A FUNCTION TO EXTRACT ALL THE HASHTAGS WE CREATE BEFORE FROM IG

def hashtagextraction(hashes,df):
    
    hashtag = []
    number = []
    counter = 0
    for i in tqdm(hashes):
        url = "https://www.instagram.com/explore/tags/"+i+"/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        hashtag.append(i)
        number.append(soup.prettify().split('posts - Discover')[0][len(soup.prettify().split(' posts - Discover')[0])-25:])
        counter += 1
        if counter == 300:
            sleep(10)
            counter = 0
    
    # I'm going to take the info I need (number of hashtags) from the java text, because I can't acces the info scraping
    num_hashtags = []
    for i in number:
        #the ones with <meta content
        if "<meta content" in i:
            #how >> split it on <meta content=\'
            num_hashtags.append(i.split("<meta content=\'")[1])
        else:
            num_hashtags.append("0")
            
            
    hashtags_final = dict(zip(hashtag,num_hashtags))
    
    
    return df['#1'].map(hashtags_final), df['#2'].map(hashtags_final)

In [12]:
#Putting the scraped info in two columns

asos['#1num'], asos['#2num']  = hashtagextraction(hashsearch,asos)

100%|█████████████████████████████████████| 5356/5356 [1:25:07<00:00,  1.05it/s]


In [15]:
#export dataframe with hashtags info

asos.to_csv('data/asos_hashtags.csv', index = False)

In [4]:
asos_hashtags = pd.read_csv('data/asos_hashtags.csv')

In [5]:
asos_hashtags

,name,price,brand,productinfo,categories,#search,#colour,#brand,#1,#2,#1num,#2num
0,New Look trench coat in camel,49.99,stradivarius,coats & jackets,coats,coat,beige,stradivarius,beigecoat,stradivariuscoat,30K,1.6K
1,New Look trench coat in camel,49.99,stradivarius,coats & jackets,coats,coat,beige,stradivarius,beigecoat,stradivariuscoat,30K,1.6K
2,New Look trench coat in camel,49.99,asos-design,coats & jackets,coats,coat,beige,asos design,beigecoat,asosdesigncoat,30K,3
3,New Look trench coat in camel,49.99,new-look,coats & jackets,coats,coat,beige,new look,beigecoat,newlookcoat,30K,888
4,Stradivarius double breasted wool coat in grey,59.99,stradivarius,coats & jackets,coats,coat,grey,stradivarius,greycoat,stradivariuscoat,36K,1.6K
...,...,...,...,...,...,...,...,...,...,...,...,...
29772,ASOS DESIGN satin midi dress with cowl neck an...,55.00,asos-design,dresses,dresses,dress,brown,asos design,browndress,asosdesigndress,68K,53
29773,Urban Revivo square neck mini dress in floral ...,44.00,urban-revivo,mini dress,dresses,minidress,pattern,urban revivo,patternminidress,urbanrevivominidress,8,0
29774,ASOS DESIGN long sleeve maxi t-shirt dress in ...,24.00,asos-design,dress,dresses,dress,black,asos design,blackdress,asosdesigndress,4.9M,53
29775,ASYOU layered t-shirt dress with focus graphic...,22.99,asyou,dress,dresses,dress,black,asyou,blackdress,asyoudress,4.9M,9


In [7]:
#CREATING A FUNCTION TO CLEAN ALL THIS NEW DATA FROM IG

def num_hashtag_cleaner(x):
    
    x = x.astype(str)
    #ISOLATING THE NUMBER OG HASHTAGS BY CHANGING K AND M FOR ZEROS
    num1_clean = []
    
    for i in x:
        #if M
        if 'K' in i:
            if "." in i:
                #remove K and . and put 00
                i = i.replace("K","")
                i = i.replace(".","")
                num1_clean.append(i+'00')
            else: 
                #delete K and put 000
                i = i.replace("K","")
                num1_clean.append(i+'000')
        #if M
        elif 'M' in i:
            if "." in i:
                #remove M and . and put 00000
                i = i.replace("M","")
                i = i.replace(".","")
                num1_clean.append(i+'00000')
            else:
                #delete M and put 000000
                i = i.replace("M","")
                num1_clean.append(i+'000000')
        else:
            # If there's no 'K' or 'M', simply append the original value
            num1_clean.append(i)
    
    x = num1_clean
    
 
    return x

In [8]:
#APPLYING HASHTAG CLEANING FUNCTION TO COLUMNS
asos_hashtags['#1num'] = num_hashtag_cleaner(asos_hashtags['#1num'])
asos_hashtags['#2num'] = num_hashtag_cleaner(asos_hashtags['#2num'])

In [9]:
#DELETING THE SPACES BETWEEN NUMBERS TO BE ABLE TO CONVERT INTO A STRING
asos_hashtags['#1num'] = asos_hashtags['#1num'].str.replace(' ','')
asos_hashtags['#1num'] = asos_hashtags['#1num'].str.replace('nan','0')
asos_hashtags['#2num'] = asos_hashtags['#2num'].str.replace(' ','')
asos_hashtags['#2num'] = asos_hashtags['#2num'].str.replace('nan','0')

In [10]:
#FILL NAN VALUES WITH A 0 - THE SCRAPER COULD NOT FIND ANY RESULTS - HASH TAG DOES NOT EXIST
asos_hashtags['#1num'] = asos_hashtags['#1num'].fillna('0')
asos_hashtags['#2num'] = asos_hashtags['#2num'].fillna('0')


In [11]:
#CHANGING TYPE OBJ TO INT
asos_hashtags['#1num'] = asos_hashtags['#1num'].astype(int)
asos_hashtags['#2num'] = asos_hashtags['#2num'].astype(int)

In [12]:
#I'M GOING TO CONVERT COLUMN 'PRICE' INTO A FLOAT AND CORRECT A COUPLE OF ERRORS
asos_hashtags['price'] = asos_hashtags['price'].str.replace('From', '')
asos_hashtags['price'] = asos_hashtags['price'].str.replace('Now', '')
asos_hashtags['price'] = asos_hashtags['price'].astype(float)

#THIS IS GOING TO BE MY SELECTED COLUMNS FOR MY MODELS
asos_hashtags = asos_hashtags[['name', 'price', '#search','#brand','categories', '#colour', '#1', '#1num','#2', '#2num']]
  
#FINALLY, I'M GOING TO CREATE A NEW COLUMN WITH THE SUM OF #1 AND #2, TO USE IT FOR MY MODEL AS MY TARGET   
asos_hashtags['potential_hashtag'] = asos_hashtags['#1num']+asos_hashtags['#2num']


/var/folders/75/lxv2pt816y3g7mh_b0zy07mw0000gn/T/ipykernel_4874/447343852.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asos_hashtags['potential_hashtag'] = asos_hashtags['#1num']+asos_hashtags['#2num']


In [14]:
asos_hashtags

,name,price,#search,#brand,categories,#colour,#1,#1num,#2,#2num,potential_hashtag
0,New Look trench coat in camel,49.99,coat,stradivarius,coats,beige,beigecoat,30000,stradivariuscoat,1600,31600
1,New Look trench coat in camel,49.99,coat,stradivarius,coats,beige,beigecoat,30000,stradivariuscoat,1600,31600
2,New Look trench coat in camel,49.99,coat,asos design,coats,beige,beigecoat,30000,asosdesigncoat,3,30003
3,New Look trench coat in camel,49.99,coat,new look,coats,beige,beigecoat,30000,newlookcoat,888,30888
4,Stradivarius double breasted wool coat in grey,59.99,coat,stradivarius,coats,grey,greycoat,36000,stradivariuscoat,1600,37600
...,...,...,...,...,...,...,...,...,...,...,...
29772,ASOS DESIGN satin midi dress with cowl neck an...,55.00,dress,asos design,dresses,brown,browndress,68000,asosdesigndress,53,68053
29773,Urban Revivo square neck mini dress in floral ...,44.00,minidress,urban revivo,dresses,pattern,patternminidress,8,urbanrevivominidress,0,8
29774,ASOS DESIGN long sleeve maxi t-shirt dress in ...,24.00,dress,asos design,dresses,black,blackdress,4900000,asosdesigndress,53,4900053
29775,ASYOU layered t-shirt dress with focus graphic...,22.99,dress,asyou,dresses,black,blackdress,4900000,asyoudress,9,4900009


In [147]:
asos_hashtags.to_csv('data/asos_clean.csv', index=False)